In [6]:
from math import floor
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

scale_number = 2

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        return np.array(img)

def save_image(after_process_img):
    img = Image.fromarray(after_process_img,mode="RGB")
    img.save(f"处理后图片{scale_number}x{scale_number}.jpg")

def bilinear_interpolation(raw_pic : np.ndarray,mul_width,mul_height):
    h, w = raw_pic.shape
    new_h,new_w = h * mul_height,w * mul_width
    scale_h = (h - 1) / (new_h - 1)
    scale_w = (w - 1) / (new_w - 1)
    result = np.zeros((new_h,new_w))
    for i in range(new_h):
        for j in range(new_w):
            y0, x0 = floor(scale_h * i),floor(scale_w * j)
            y1, x1 = min(y0 + 1,h - 1),min(x0 + 1,w - 1)
            dy = scale_h * i - y0
            dx = scale_w * j - x0
            w11,w01,w10,w00 = dy * dx,(1 - dy) * dx,dy * (1 - dx),(1 - dy) * (1 - dx)
            result[i,j] = w11 * raw_pic[y1, x1] + w01 * raw_pic[y0, x1] + w10 * raw_pic[y1, x0] + w00 * raw_pic[y0, x0]
    return result


arr = load_image("原图.jpg")
channels = [arr[:,:,0],arr[:,:,1],arr[:,:,2]]
result = []
for ch in channels:
    result.append(bilinear_interpolation(ch,scale_number,scale_number))
result = np.dstack(result)
result = np.clip(result, 0, 255).astype(np.uint8)
save_image(result)

In [7]:
from scipy.ndimage import zoom

def save_image(after_process_img):
    img = Image.fromarray(after_process_img, mode="RGB")
    img.save(f"处理后图片_scipy{scale_number}x{scale_number}.jpg")

arr = load_image("原图.jpg")
result = zoom(arr, (scale_number,scale_number, 1), order=1)
result = np.clip(result, 0, 255).astype(np.uint8)
save_image(result)